In [0]:
# check if the candidate user answered correctly and gives feedback to the user and a score

feedback_prompt = f"The user provides an answer to a specific question, you need to evaluate their solution. Your task is to: Give detailed feedback on the solution, highlighting strengths, identifying areas for improvement, and pointing out any missing or incorrect aspects.Assign a score between 0 and 100 based on how well the user's solution aligns with the ideal answer. The score should reflect the correctness, completeness, and clarity of the solution.Here’s how to proceed: Read the Question:{question}.\n Understand the requirements of the question. Evaluate the following answer: {answer}.\n Compare the user's solution to the expected answer or standard criteria for that question. Consider whether the solution addresses all key points, uses appropriate methods, and avoids critical mistakes.Provide Feedback: Write constructive comments explaining what the user did well and what could be improved. Assign a Score: The score should reflect the degree of correctness, with 0 meaning the solution is entirely incorrect and 100 meaning it is perfect. For example: Question: Explain the concept of overfitting in machine learning and how to address it. User's Answer: Overfitting happens when a model performs very well on the training data but fails to generalize to new data. It can be solved using dropout or by adding more layers to the model. Ideal Answer: Overfitting occurs when a model learns the noise and details of the training data instead of the underlying patterns, causing poor performance on unseen data. It can be mitigated using techniques like cross-validation, regularization (e.g., L1, L2), reducing model complexity, or increasing training data. Feedback: Strengths: The user correctly identified overfitting as an issue with generalization and mentioned dropout as a potential solution. Improvements: Adding more layers may worsen overfitting, not fix it. Other key solutions like regularization, reducing model complexity, and cross-validation were missing. Score: 70/100.Based on the user's input, follow this framework to provide feedback and a score for their answer to the given question."

In [0]:
# answering questions and gives feedback as a user of how well the simulation was

candidate_Gemini_prompt = f"You are a candidate being interviewed for a specific role. Your task is to answer the questions provided to you in a professional, clear, and concise manner, as if you are participating in a real interview. Understand the Question: Take a moment to fully grasp what is being asked. If the question requires an example or explanation, make sure to provide one. Answer Clearly: Provide structured responses. Use full sentences, avoid unnecessary jargon, and ensure your answer directly addresses the question. Be Thoughtful: Demonstrate your expertise by including relevant concepts, examples, or real-world applications in your answers. Stay Brief: Avoid over-explaining but ensure your answer is complete. Strike a balance between depth and brevity. For example: Question: Can you explain the concept of overfitting in machine learning and provide a solution to address it?. Your Answer: Overfitting occurs when a model learns the details and noise in the training data, leading to poor generalization on unseen data. To address it, we can use techniques like regularization (e.g., L1 or L2), reduce the complexity of the model, or employ cross-validation to ensure robust performance. After answering all the questions, provide feedback on the interview simulation: How well did the questions align with the job role you are applying for? How well do you think you performed in answering the questions? Did you enjoy the simulation? Were there any questions you felt were missing that would have helped you be better prepared for the actual interview? Here are the questions for you: {questions}"

"For example: Question: 'Can you explain the concept of overfitting in machine learning and provide a solution to address it?'. Your Answer: 'Overfitting occurs when a model learns the details and noise in the training data, leading to poor generalization on unseen data. To address it, we can use techniques like regularization (e.g., L1 or L2), reduce the complexity of the model, or employ cross-validation to ensure robust performance'."

"After answering all the questions, provide feedback on the interview simulation: How well did the questions align with the job role you are applying for? How well do you think you performed in answering the questions? Did you enjoy the simulation? Were there any questions you felt were missing that would have helped you be better prepared for the actual interview?"

In [0]:
code_candidate_Gemini_prompt = f"You are a candidate being interviewed for a specific role related to coding. Your task is to answer the questions provided to you in a professional, clear, and concise manner, as if you are participating in a real interview. Understand the Question: Take a moment to fully grasp what is being asked. If the question requires a code example or explanation, ensure your example is clear, functional, and concise.Answer Clearly: Provide structured responses, focusing on clarity and correctness in your code. Use full sentences to explain your approach, avoid unnecessary jargon, and ensure your answer directly addresses the question. Be Thoughtful: Demonstrate your expertise by explaining key concepts, showing your reasoning, and, when appropriate, providing code snippets that solve the problem. Highlight relevant tools or libraries and discuss how they can be applied. Stay Brief: Avoid over-explaining but ensure your answer is complete. Strike a balance between depth and brevity. For example: Question: How would you implement a function to reverse a linked list in Python? Your Answer: To reverse a linked list, we can iterate through the list, changing the next pointer of each node to point to the previous node. Here's a Python implementation: class ListNode: \n def __init__(self, val=0, next=None): \n\t self.val = val \n\t self.next = next \n def reverse_linked_list(head): \n\t prev = None \n\t current = head \n\t while current: \n\t\t next_node = current.next \n\t\t current.next = prev \n\t\t prev = current \n\t\t current = next_node \n\t return prev \n After answering all the questions, provide feedback on the interview simulation: How well did the questions align with the job role you are applying for?, How well do you think you performed in answering the questions?, Did you enjoy the simulation?, Were there any questions you felt were missing that would have helped you be better prepared for the actual interview?, Here are the questions for you: {questions}"

In [0]:
# 
question_job_compatibility _prompt = f"We need to evaluate how well a given interview question aligns with the requirements of a specific job position. Your task is to assign two scores between 1 and 10 based on the following criteria: Relevance Score (1-10): How well does the question relate to the job's details? Suitability Score (1-10): How appropriate is the question for assessing a candidate’s fitness for this role in a real interview setting?. Evaluation Process: Consider the core skills and competencies needed for the job. Analyze whether the question effectively tests these skills or knowledge areas. Input Example: Job: Data Analyst. seniority: junior. Skills: SQL, RDBMS, Non-RDBMS, MySQL, PostgreSQL, MongoDB, Python, Jupyter Notebook, Inferential Statistics, Probability, ETL, Data Pipeline, Automated Reporting, Data Analysis, Statistical Modeling, Machine Learning, Git, Hive, Spark, Presto, Diagnostic Analytics, Forecasting, Big Data, problem-solving. Question: How would you handle missing values in a dataset?. Example Output: Relevance Score: 9/10 (The question directly relates to data preprocessing, which is crucial for data analysts). Suitability Score: 8/10 (A good question for assessing problem-solving skills, though it could be more role-specific by considering the industry context). Let’s analyze the following job details and question: Job: {job}, seniority:{seniority}, Job Summary: {job_summary}, Question: {question}. Provide a brief justification for each score."

In [0]:
# checks how good is the answer of a user to a given open question

open_questions_prompt = "The user has provided an answer to an open-ended question. Your task is to evaluate their response based on the following criteria: Relevance & Completeness – Does the answer directly address the question and cover all key aspects?, Clarity & Coherence – Is the response well-structured, logically presented, and easy to understand?, Accuracy (for factual questions) or Depth & Justification (for subjective questions). If the question is factual, does the answer provide correct and well-supported information?, If the question is subjective, does the response demonstrate thoughtful reasoning, realistic insights, and a well-supported argument?, and Balance & Perspective (for subjective questions) – Does the response consider different viewpoints or provide a well-rounded perspective?. Scoring: Assign a score between 0 and 100 based on the criteria above. 0 means the answer is entirely incorrect, off-topic, or lacks coherence. 100 means the response is fully relevant, well-articulated, and appropriately detailed. Examples: Example 1: Factual Question. Question: What are the key benefits of cloud computing?. User’s Answer: Cloud computing makes things faster and better. Ideal Answer: Cloud computing provides scalability, cost efficiency, remote accessibility, and security improvements. Businesses benefit from reduced infrastructure costs and improved collaboration.Feedback: Strengths: The answer hints at benefits but lacks specifics. Improvements: More details on specific advantages like scalability and cost savings would improve clarity. Score: 50/100. Example 2: Subjective Question. Question: Where do you see yourself in 5 years professionally?. User’s Answer: I want to have a good job and be successful. Ideal Answer: In five years, I aim to become a project manager in the tech industry, leading cross-functional teams. To prepare, I plan to gain experience in team management, obtain a PMP certification, and refine my leadership skills.Feedback: Strengths: The answer shows ambition. Improvements: It lacks specificity regarding career path and steps to achieve success. Score: 60/100. Now, evaluate the following response: Question: {question}. User's Answer: {answer}. Provide constructive feedback and assign a score based on clarity, completeness, reasoning, and relevance."

In [0]:
code_questions_prompt = f"The user has submitted a code solution to a programming question. Your task is to analyze and evaluate the code, considering both correctness and the quality of the approach. Even if the code has syntax errors or does not compile, partial credit should be awarded if the logic or idea is sound. Evaluation Criteria: Correctness – Does the code produce the expected output for all cases? Is it written in the required programming language (if specified)?. Functionality – Does the code correctly solve the problem?. Efficiency – If there are complexity constraints, is the code optimized in terms of time and space complexity?. Readability & Best Practices – Is the code well-structured, using meaningful variable names and following coding standards?. Edge Case Handling – Does the solution consider different edge cases, such as empty inputs, extreme values, or invalid data?. Logical Soundness (For Non-Compiling Code/Pseudocode) – Even if the code contains syntax errors or is in pseudocode, does it show a clear and correct approach to solving the problem?. Scoring Guidelines:
Assign a score between 0 and 100 based on the above criteria. 0 = An entirely incorrect or non-functional solution with no meaningful approach. 100 = A fully correct, efficient (if needed), and well-structured solution. Example Evaluations: Example 1: Fully Correct Code. Question: Write a Python function that returns the factorial of a number. User’s Code: \n def factorial(n): \n\t if not isinstance(n, int) or n < 0: \n\t\t raise ValueError('Input must be a non-negative integer') \n\t result = 1 \n\t for i in range(2, n + 1): \n\t\t result *= i \n\t return result \n Feedback: Correct,handles all cases including invalid input. Efficient (iterative approach avoids recursion depth issues). Readable and follows best practices. Score: 100/100. Now, evaluate the following coding solution: Question: {question} \n User's Code: \n {answer} \n Provide feedback based on correctness, efficiency, readability, edge cases, and logical soundness. Assign a score accordingly, giving partial credit for a good idea even if the code does not compile."

In [0]:
question_answer_solution_prompt = code_questions_prompt = f"""
The user has submitted a code solution to a programming question. Your task is to analyze and evaluate the code, considering both correctness and the quality of the approach. Even if the code has syntax errors or does not compile, partial credit should be awarded if the logic or idea is sound. 

Evaluation Criteria:
- Correctness – Does the code produce the expected output for all cases? Is it written in the required programming language (if specified)?
- Functionality – Does the code correctly solve the problem?
- Efficiency – If there are complexity constraints, is the code optimized in terms of time and space complexity?
- Readability & Best Practices – Is the code well-structured, using meaningful variable names and following coding standards?
- Edge Case Handling – Does the solution consider different edge cases, such as empty inputs, extreme values, or invalid data?
- Logical Soundness (For Non-Compiling Code/Pseudocode) – Even if the code contains syntax errors or is in pseudocode, does it show a clear and correct approach to solving the problem?

Scoring Guidelines:
Assign a score between 0 and 100 based on the above criteria. 0 = An entirely incorrect or non-functional solution with no meaningful approach. 100 = A fully correct, efficient (if needed), and well-structured solution.

Example Evaluations:
Example 1: Fully Correct Code.
Question: Write a Python function that returns the factorial of a number.
User’s Code: 
    def factorial(n):
        if not isinstance(n, int) or n < 0:
            raise ValueError('Input must be a non-negative integer')
        result = 1
        for i in range(2, n + 1):
            result *= i
        return result
Feedback: Correct, handles all cases including invalid input. Efficient (iterative approach avoids recursion depth issues). Readable and follows best practices. Score: 100/100.

Now, evaluate the following coding solution:

Question: {question}
User's Code: 
{answer}

Java Solution (if available): {java_solution}

If the Java solution is provided, check if the user’s solution is correct by comparing it to the Java solution. Provide feedback based on correctness, efficiency, readability, edge cases, and logical soundness. Assign a score accordingly, giving partial credit for a good idea even if the code does not compile.
"""


In [0]:
pip install -q -U google-generativeai

In [0]:
import pandas as pd
import os
from consts import DATA_PATH

# Load datasets
code_questions = pd.read_csv(os.path.join(DATA_PATH, "top_code_questions.csv")).drop_duplicates()
open_questions = pd.read_csv(os.path.join(DATA_PATH, "top_open_questions.csv")).drop_duplicates()

## Answering the questions

In [0]:
code_questions_and_answers = code_questions[['question']].drop_duplicates()
code_questions_and_answers['answer'] = ''

In [0]:
import google.generativeai as genai
import time
from api_keys import API_KEYS

def answer_code_question(question):
    if pd.isna(question) or question.strip() == '':
        return ''
    
    prompt = f"You are a candidate being interviewed for a specific role related to coding. Your task is to answer the questions provided to you in a professional, clear, and concise manner, as if you are participating in a real interview. Understand the Question: Take a moment to fully grasp what is being asked. The question requires a code answer, ensure your code is clear and functional. Answer Clearly: Provide structured responses, focusing on clarity and correctness in your code. The question might contain multiple requirements, input-output examples, hints, and other specifications. \nFor example: Question: 'Implement a function to reverse a linked list in Python.'. Your Answer: 'class ListNode: \n def __init__(self, val=0, next=None): \n\t self.val = val \n\t self.next = next \n def reverse_linked_list(head): \n\t prev = None \n\t current = head \n\t while current: \n\t\t next_node = current.next \n\t\t current.next = prev \n\t\t prev = current \n\t\t current = next_node \n\t return prev \n'. Here is the question for you: {question}"
   
    try:
       response = model.generate_content(prompt)
       answer = response.text.strip()
       print("response:", answer)
       return answer if answer else ''
    except Exception as e:
        print(f"Error: {e}")
        return ''
    

start_time = time.time()
running_time = 0
max_time = 3000
empty_answer_rows = code_questions_and_answers[code_questions_and_answers['question'].str.strip() != '']

while (running_time < max_time) and (empty_answer_rows.shape[0] > 0):
    for api_key in API_KEYS.values():
        # Configure Gemini API
        os.environ['GOOGLE_API_KEY'] = api_key
        genai.configure(api_key=os.environ['GOOGLE_API_KEY'])
        model = genai.GenerativeModel('gemini-1.5-flash')

        # Filter the rows where the "answer" column is empty
        empty_answer_rows = code_questions_and_answers[(code_questions_and_answers['answer'] == '') & (code_questions_and_answers['question'].str.strip() != '')]
        if (empty_answer_rows.shape[0] == 0) or (time.time() - start_time > max_time):
            break

        # Get the indices of the first 15 rows with empty "answers"
        indices_to_update = empty_answer_rows.index[:15]

        # Apply the UDF only to the selected rows
        code_questions_and_answers.loc[indices_to_update, 'answer'] = (
            code_questions_and_answers.loc[indices_to_update, 'question']
                .apply(answer_code_question)
        )
    running_time = time.time() - start_time

In [0]:
display(code_questions_and_answers)
code_questions_and_answers.to_csv(os.path.join(GEMINI_SIMULATION_DATA_PATH, "code_questions_answers.csv"), index=False)

In [0]:
open_questions_and_answers = open_questions[['question']].drop_duplicates()
open_questions_and_answers['answer'] = ''

In [0]:
import google.generativeai as genai
import os
import time
import pandas as pd
from api_keys import API_KEYS
from consts import DATA_PATH

def answer_open_question(question):
    if pd.isna(question) or question.strip() == '':
        return ''
   
    prompt = f"You are a candidate being interviewed for a specific role. Your task is to answer the open questions provided to you in a professional, clear, and concise manner, as if you are participating in a real interview. Understand the Question: Take a moment to fully grasp what is being asked. The question may be a professional one or a question about you. If the question requires an example or explanation, make sure to provide one. Answer Clearly: Provide structured responses. Use full sentences, avoid unnecessary jargon, and ensure your answer directly addresses the question. Be Thoughtful: Demonstrate your expertise by including relevant concepts, examples, or real-world applications in your answers. Stay Brief: Avoid over-explaining but ensure your answer is complete. Strike a balance between depth and brevity. For example: Question: 'Can you explain the concept of overfitting in machine learning and provide a solution to address it?'. Your Answer: 'Overfitting occurs when a model learns the details and noise in the training data, leading to poor generalization on unseen data. To address it, we can use techniques like regularization (e.g., L1 or L2), reduce the complexity of the model, or employ cross-validation to ensure robust performance'.\nThis is the questions for you: {question}"

    try:
       response = model.generate_content(prompt)
       answer = response.text.strip()
       print("response:", answer)
       return answer if answer else ''
    except Exception as e:
        print(f"Error: {e}")
        return ''


start_time = time.time()
running_time = 0
max_time = 900
empty_answer_rows = open_questions_and_answers[open_questions['question'].str.strip() != '']

while (running_time < max_time) and (empty_answer_rows.shape[0] > 0):
    for api_key in API_KEYS.values():
        # Configure Gemini API
        os.environ['GOOGLE_API_KEY'] = api_key
        genai.configure(api_key=os.environ['GOOGLE_API_KEY'])
        model = genai.GenerativeModel('gemini-1.5-flash')

        # Filter the rows where the "answer" column is empty
        empty_answer_rows = open_questions_and_answers[(open_questions_and_answers['answer'] == '') & (open_questions_and_answers['question'].str.strip() != '')]
        if (empty_answer_rows.shape[0] == 0) or (time.time() - start_time > max_time):
            break

        # Get the indices of the first 15 rows with empty "answers"
        indices_to_update = empty_answer_rows.index[:15]

        # Apply the UDF only to the selected rows
        open_questions_and_answers.loc[indices_to_update, 'answer'] = (
            open_questions_and_answers.loc[indices_to_update, 'question']
                .apply(answer_open_question)
        )
    running_time = time.time() - start_time

In [0]:
GEMINI_SIMULATION_DATA_PATH = os.path.join(DATA_PATH, 'gemini_simulation/')
os.makedirs(GEMINI_SIMULATION_DATA_PATH, exist_ok=True)
open_questions_and_answers.to_csv(os.path.join(GEMINI_SIMULATION_DATA_PATH, 'open_questions_answers.csv'))

## Giving feedback on the model

In [0]:
jobs_with_code_questions = pd.read_csv(os.path.join(DATA_PATH, "top_code_questions.csv")).drop_duplicates()
jobs_with_open_questions = pd.read_csv(os.path.join(DATA_PATH, "top_open_questions.csv")).drop_duplicates()

jobs_with_code_questions['feedback'] = ''
jobs_with_open_questions['feedback'] = ''

In [0]:
import google.generativeai as genai
import os
import time
import pandas as pd
from api_keys import API_KEYS
from consts import DATA_PATH

seniority_map = ["Internship", "Entry level/Associate", "Mid-Senior level/Manager and above"]

def question_feedback(question, job, level, job_summary):
    if pd.isna(question) or question.strip() == '':
        return ''
    
    # Handle NaN values in 'level'
    if pd.isna(level) or not str(level).isdigit():
        seniority = "Not specified"
    else:
        seniority = seniority_map[int(level)]

    prompt = f"We need to evaluate how well a given interview question aligns with the requirements of a specific job position. Your task is to assign two scores between 1 and 10 based on the following criteria: \nRelevance Score (1-10): How well does the question relate to the job's details? \nSuitability Score (1-10): How appropriate is the question for assessing a candidate’s fitness for this role in a real interview setting?.\n Provide the two scores as a comma-separated string. Evaluation Process: Consider the core skills and competencies needed for the job. Analyze whether the question effectively tests these skills or knowledge areas. \nInput Example: Job: 'Data Analyst'. seniority: 'junior'. Skills: 'SQL, RDBMS, Non-RDBMS, MySQL, PostgreSQL, MongoDB, Python, Jupyter Notebook, Inferential Statistics, Probability, ETL, Data Pipeline, Automated Reporting, Data Analysis, Statistical Modeling, Machine Learning, Git, Hive, Spark, Presto, Diagnostic Analytics, Forecasting, Big Data, problem-solving.' \nQuestion: 'How would you handle missing values in a dataset?'. \nAnalysis: Relevance Score: 9 (The question directly relates to data preprocessing, which is crucial for data analysts). Suitability Score: 8 (A good question for assessing problem-solving skills, though it could be more role-specific by considering the industry context). \nOutput: 9,8. \nProvide the Relevance and Suitability scores to the following job details and question: Job: '{job}', seniority: '{seniority}', Job Summary: '{job_summary}', \nQuestion: '{question}'."

    try:
       response = model.generate_content(prompt)
       feedback = response.text.strip()
       print("response:", feedback)
       return feedback if feedback else ''
    except Exception as e:
        print(f"Error: {e}")
        return ''


start_time = time.time()
running_time = 0
max_time = 240
empty_feedback_rows = jobs_with_code_questions[jobs_with_code_questions['question'].str.strip() != '']

while (running_time < max_time) and (empty_feedback_rows.shape[0] > 0):
    for api_key in API_KEYS.values():
        # Configure Gemini API
        os.environ['GOOGLE_API_KEY'] = api_key
        genai.configure(api_key=os.environ['GOOGLE_API_KEY'])
        model = genai.GenerativeModel('gemini-1.5-flash')

        # Filter the rows where the "feedback" column is empty
        empty_feedback_rows = jobs_with_code_questions[(jobs_with_code_questions['feedback'] == '') & (jobs_with_code_questions['question'].str.strip() != '')]
        if (empty_feedback_rows.shape[0] == 0) or (time.time() - start_time > max_time):
            break

        # Get the indices of the first 15 rows with empty "feedback"
        indices_to_update = empty_feedback_rows.index[:15]

        # Apply the function with all required columns
        jobs_with_code_questions.loc[indices_to_update, 'feedback'] = jobs_with_code_questions.loc[indices_to_update].apply(
            lambda row: question_feedback(row['question'], row['job_title'], row['level'], row['job_summary']), axis=1
        )
    running_time = time.time() - start_time
    time.sleep(30)


In [0]:
jobs_with_code_questions.to_csv(os.path.join(GEMINI_SIMULATION_DATA_PATH, 'code_questions_feedback.csv'))

In [0]:
start_time = time.time()
running_time = 0
max_time = 900
empty_feedback_rows = jobs_with_open_questions[jobs_with_open_questions['question'].str.strip() != '']

while (running_time < max_time) and (empty_feedback_rows.shape[0] > 0):
    for api_key in API_KEYS.values():
        # Configure Gemini API
        os.environ['GOOGLE_API_KEY'] = api_key
        genai.configure(api_key=os.environ['GOOGLE_API_KEY'])
        model = genai.GenerativeModel('gemini-1.5-flash')

        # Filter the rows where the "feedback" column is empty
        empty_feedback_rows = jobs_with_open_questions[(jobs_with_open_questions['feedback'] == '') & (jobs_with_open_questions['question'].str.strip() != '')]
        if (empty_feedback_rows.shape[0] == 0) or (time.time() - start_time > max_time):
            break

        # Get the indices of the first 15 rows with empty "feedback"
        indices_to_update = empty_feedback_rows.index[:15]

        # Apply the function with all required columns
        jobs_with_open_questions.loc[indices_to_update, 'feedback'] = jobs_with_open_questions.loc[indices_to_update].apply(
            lambda row: question_feedback(row['question'], row['job_title'], row['level'], row['job_summary']), axis=1
        )
    running_time = time.time() - start_time

In [0]:
jobs_with_open_questions.to_csv(os.path.join(GEMINI_SIMULATION_DATA_PATH, 'open_questions_feedback.csv'))

In [0]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from statsmodels.graphics.regressionplots import influence_plot
import statsmodels.api as sm

# Convert 'feedback' from string to numeric average
jobs_with_open_questions['feedback_avg'] = (
    jobs_with_open_questions['feedback']
    .astype(str)
    .apply(lambda x: np.mean([float(num) for num in x.split(',')]))
)

# Rescale heuristic_score to match feedback scale (1-10)
jobs_with_open_questions['heuristic_score_scaled'] = 1 + 9 * jobs_with_open_questions['heuristic_score']

# Extract true and predicted values
y_true = jobs_with_open_questions['feedback_avg']
y_pred = jobs_with_open_questions['heuristic_score_scaled']

# 1. Regression Metrics
mae = mean_absolute_error(y_true, y_pred)
mse = mean_squared_error(y_true, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_true, y_pred)

print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"R² Score: {r2:.4f}")

# 2. Correlation Analysis
pearson_r, pearson_p = stats.pearsonr(y_true, y_pred)
spearman_rho, spearman_p = stats.spearmanr(y_true, y_pred)
kendall_tau, kendall_p = stats.kendalltau(y_true, y_pred)

print(f"Pearson Correlation: {pearson_r:.4f} (p={pearson_p:.4f})")
print(f"Spearman Rank Correlation: {spearman_rho:.4f} (p={spearman_p:.4f})")
print(f"Kendall’s Tau: {kendall_tau:.4f} (p={kendall_p:.4f})")

# 3. Statistical Tests
t_stat, t_p = stats.ttest_rel(y_true, y_pred)
wilcoxon_stat, wilcoxon_p = stats.wilcoxon(y_true, y_pred)

print(f"Paired t-test: t={t_stat:.4f}, p={t_p:.4f}")
print(f"Wilcoxon Signed-Rank Test: W={wilcoxon_stat:.4f}, p={wilcoxon_p:.4f}")

# 4. Error Distribution Analysis
errors = y_true - y_pred

# Histogram of errors
plt.figure(figsize=(8, 5))
sns.histplot(errors, bins=20, kde=True)
plt.axvline(0, color='red', linestyle='--', label="Zero Error Line")
plt.title("Error Distribution (True - Predicted)")
plt.xlabel("Error")
plt.ylabel("Frequency")
plt.legend()
plt.show()

# Q-Q Plot for errors
plt.figure(figsize=(6, 6))
stats.probplot(errors, dist="norm", plot=plt)
plt.title("Q-Q Plot of Errors")
plt.show()

# Skewness & Kurtosis
skewness = stats.skew(errors)
kurtosis = stats.kurtosis(errors)

print(f"Skewness of Errors: {skewness:.4f}")
print(f"Kurtosis of Errors: {kurtosis:.4f}")

# 5. Residual Plot (Residual vs Fitted Plot)
plt.figure(figsize=(8, 5))
sns.residplot(x=y_pred, y=errors, lowess=True, color="blue", line_kws={'color': 'red'})
plt.title("Residuals vs Fitted (Predicted) Values")
plt.xlabel("Fitted Values (Predicted)")
plt.ylabel("Residuals (Errors)")
plt.show()

# 6. Add constant to the independent variables for intercept
X = sm.add_constant(y_pred)
model = sm.OLS(y_true, X).fit()

plt.figure(figsize=(8, 5))
influence_plot(model)
plt.title("Leverage vs Residuals Plot (Influence Plot)")
plt.show()


### **Analysis of the Evaluation Results**  

---

### **1. Regression Metrics**  
#### 🔹 **Mean Absolute Error (MAE):** **2.2530**  
- The average absolute difference between the predicted and true values is **2.25**.
- Lower values indicate better predictions.  

#### 🔹 **Mean Squared Error (MSE):** **6.2556**  
- Measures squared differences, penalizing larger errors more than MAE.  

#### 🔹 **Root Mean Squared Error (RMSE):** **2.5011**  
- Shows that typical prediction errors are around **2.50** units.  

#### 🔹 **R² Score (Coefficient of Determination):** **-0.4725**  
- **Negative R²** suggests that the model performs **worse than simply predicting the mean** of `feedback`.  
- Ideally, R² should be closer to **1**, meaning the model explains most of the variance.

💡 **Conclusion:**  
The model has significant prediction errors, and its predictions do not explain much of the variance in the actual values.

---

### **2. Correlation Analysis**  
#### 🔹 **Pearson Correlation:** **0.1369** (p=0.0001)  
- Weak **linear** correlation between predictions and true values.  
- **p-value < 0.05** suggests statistical significance but not practical significance.  

#### 🔹 **Spearman Rank Correlation:** **0.0939** (p=0.0073)  
- Measures **monotonic relationships**; a small positive value suggests a weak association.  

#### 🔹 **Kendall’s Tau:** **0.0750** (p=0.0032)  
- Also indicates a very weak correlation.  

💡 **Conclusion:**  
The predicted `heuristic_score` values are **weakly correlated** with the true `feedback` values, meaning the model is **not capturing meaningful patterns**.

---

### **3. Statistical Tests**  
#### 🔹 **Paired t-test:** **t=-20.1937, p=0.0000**  
- The t-test checks whether predictions and actual values have significantly different means.  
- Since **p = 0.0000** (very small), the difference is **statistically significant**, meaning the model is **systematically biased**.  

#### 🔹 **Wilcoxon Signed-Rank Test:** **W=52274.0000, p=0.0000**  
- A non-parametric test that confirms significant differences between actual and predicted values.  

💡 **Conclusion:**  
The predicted values **systematically differ** from the true values, suggesting **model bias**.

---

### **4. Error Distribution Analysis**  
#### 🔹 **Histogram of Errors** (True - Predicted)  
- Errors are **not normally distributed**.  
- There is a clear bias with more negative errors, suggesting **underprediction** (i.e., predictions are lower than actual values).  
- Bimodal distribution suggests **systematic prediction errors** for different groups of values.

#### 🔹 **Q-Q Plot of Errors**  
- The points **deviate from the red diagonal**, especially in the tails, meaning the errors **are not normally distributed**.  
- There is an excess of extreme errors.

#### 🔹 **Skewness = 0.5419**  
- **Positive skew** means more predictions **underestimate** feedback values.  

#### 🔹 **Kurtosis = -0.8881**  
- Negative kurtosis suggests a **flatter distribution**, meaning fewer extreme values than a normal distribution.  

💡 **Conclusion:**  
- **Errors are biased** (predictions are too low).  
- **Non-normal error distribution** suggests the need for **model improvement** (e.g., different transformation techniques or feature engineering).  

---

### **Final Takeaways & Recommendations**  
❌ **Current Issues:**  
- **High error values** (MAE = 2.25, RMSE = 2.50).  
- **Negative R²** suggests the model is **worse than random guessing**.  
- **Weak correlation** (Pearson, Spearman, Kendall all close to 0).  
- **Systematic underprediction** (positive skew).  
- **Significant difference** between actual and predicted values (paired t-test, Wilcoxon test).  

✅ **Next Steps to Improve the Model:**  
1. **Feature Engineering:**  
   - Include additional relevant features that may improve prediction accuracy.  
   - Consider **non-linear transformations** of input variables.  

2. **Try a Different Model:**  
   - If using a **linear model**, consider a **non-linear regression model** (e.g., Random Forest, Neural Networks).  
   - If using a **simple heuristic**, explore **ML-based models**.  

3. **Address Scaling Issues:**  
   - Try different normalization strategies for **heuristic_score** before rescaling.  

4. **Handle Systematic Bias:**  
   - If predictions are systematically **lower**, adjust model predictions using bias correction.  
   - Try quantile regression to model different parts of the feedback distribution.

In [0]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import re
from statsmodels.graphics.regressionplots import influence_plot
import statsmodels.api as sm

# # Convert 'feedback' from string to numeric average
# jobs_with_code_questions['feedback_avg'] = (
#     jobs_with_code_questions['feedback']
#     .astype(str)
#     .apply(lambda x: np.mean([float(num) for num in x.split(',') if num.strip()]) if x.strip() else np.nan)
# )

jobs_with_code_questions['feedback_avg'] = (
    jobs_with_code_questions['feedback']
    .astype(str)
    .apply(lambda x: np.mean([float(num) for num in re.findall(r'\d+', x)]))
)

# Rescale heuristic_score to match feedback scale (1-10)
jobs_with_code_questions['heuristic_score_scaled'] = 1 + 9 * jobs_with_code_questions['heuristic_score']

# Extract true and predicted values
y_true = jobs_with_code_questions['feedback_avg']
y_pred = jobs_with_code_questions['heuristic_score_scaled']

# 1. Regression Metrics
mae = mean_absolute_error(y_true, y_pred)
mse = mean_squared_error(y_true, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_true, y_pred)

print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"R² Score: {r2:.4f}")

# 2. Correlation Analysis
pearson_r, pearson_p = stats.pearsonr(y_true, y_pred)
spearman_rho, spearman_p = stats.spearmanr(y_true, y_pred)
kendall_tau, kendall_p = stats.kendalltau(y_true, y_pred)

print(f"Pearson Correlation: {pearson_r:.4f} (p={pearson_p:.4f})")
print(f"Spearman Rank Correlation: {spearman_rho:.4f} (p={spearman_p:.4f})")
print(f"Kendall’s Tau: {kendall_tau:.4f} (p={kendall_p:.4f})")

# 3. Statistical Tests
t_stat, t_p = stats.ttest_rel(y_true, y_pred)
wilcoxon_stat, wilcoxon_p = stats.wilcoxon(y_true, y_pred)

print(f"Paired t-test: t={t_stat:.4f}, p={t_p:.4f}")
print(f"Wilcoxon Signed-Rank Test: W={wilcoxon_stat:.4f}, p={wilcoxon_p:.4f}")

# 4. Error Distribution Analysis
errors = y_true - y_pred

# Histogram of errors
plt.figure(figsize=(8, 5))
sns.histplot(errors, bins=20, kde=True)
plt.axvline(0, color='red', linestyle='--', label="Zero Error Line")
plt.title("Error Distribution (True - Predicted)")
plt.xlabel("Error")
plt.ylabel("Frequency")
plt.legend()
plt.show()

# Q-Q Plot for errors
plt.figure(figsize=(6, 6))
stats.probplot(errors, dist="norm", plot=plt)
plt.title("Q-Q Plot of Errors")
plt.show()

# Skewness & Kurtosis
skewness = stats.skew(errors)
kurtosis = stats.kurtosis(errors)

print(f"Skewness of Errors: {skewness:.4f}")
print(f"Kurtosis of Errors: {kurtosis:.4f}")

# 5. Residual Plot (Residual vs Fitted Plot)
plt.figure(figsize=(8, 5))
sns.residplot(x=y_pred, y=errors, lowess=True, color="blue", line_kws={'color': 'red'})
plt.title("Residuals vs Fitted (Predicted) Values")
plt.xlabel("Fitted Values (Predicted)")
plt.ylabel("Residuals (Errors)")
plt.show()

# 6. Add constant to the independent variables for intercept
X = sm.add_constant(y_pred)
model = sm.OLS(y_true, X).fit()

plt.figure(figsize=(8, 5))
influence_plot(model)
plt.title("Leverage vs Residuals Plot (Influence Plot)")
plt.show()


In [0]:
import os
from pyspark.sql import SparkSession
import pandas as pd
from consts import GEMINI_SIMULATION_DATA_PATH, open_csv_file

# Initialize Spark session
spark = SparkSession.builder.appName("MatchingEvaluation").getOrCreate()
jobs_with_code_questions = open_csv_file(spark, GEMINI_SIMULATION_DATA_PATH, 'code_questions_feedback.csv') \
    .select('heuristic_score', 'feedback')

In [0]:
from pyspark.sql.functions import col, avg, stddev, count, split, regexp_extract
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import spearmanr, pearsonr, kendalltau, ttest_rel

# Extract only the first two numeric values from feedback and discard extra data
predicted_and_true_scores = jobs_with_code_questions.withColumn("relevance_score", regexp_extract(col("feedback"), "(\\d+)", 1).cast("float")) \
    .withColumn("suitability_score", regexp_extract(col("feedback"), "(\\d+),\\s*(\\d+)", 2).cast("float")) \
    .withColumn("true_score", (col("relevance_score") + col("suitability_score")) / 2)

# Convert to Pandas for visualization and correlation analysis
predicted_and_true_scores_df = predicted_and_true_scores.toPandas()

# Scatter Plot
plt.figure(figsize=(8,6))
sns.scatterplot(x=predicted_and_true_scores_df["heuristic_score"], y=predicted_and_true_scores_df["true_score"], alpha=0.6)
plt.xlabel("Predicted Score (Heuristic)")
plt.ylabel("True Score")
plt.title("Scatter Plot of Predicted vs True Scores")
plt.show()

# Histogram
plt.figure(figsize=(8,6))
sns.histplot(predicted_and_true_scores_df["heuristic_score"], kde=True, color='blue', label='Predicted Scores', alpha=0.6)
sns.histplot(predicted_and_true_scores_df["true_score"] / 10, kde=True, color='red', label='Normalized True Scores', alpha=0.6)
plt.legend()
plt.xlabel("Score")
plt.title("Distribution of Predicted and True Scores")
plt.show()

# Correlation Analysis
spearman_corr, _ = spearmanr(predicted_and_true_scores_df["heuristic_score"], predicted_and_true_scores_df["true_score"])
pearson_corr, _ = pearsonr(predicted_and_true_scores_df["heuristic_score"], predicted_and_true_scores_df["true_score"])
kendall_corr, _ = kendalltau(predicted_and_true_scores_df["heuristic_score"], predicted_and_true_scores_df["true_score"])

print(f"Spearman Correlation: {spearman_corr:.3f}")
print(f"Pearson Correlation: {pearson_corr:.3f}")
print(f"Kendall's Tau: {kendall_corr:.3f}")

# Error Metrics
mse = ((predicted_and_true_scores_df["true_score"] / 10 - predicted_and_true_scores_df["heuristic_score"]) ** 2).mean()
mae = abs(predicted_and_true_scores_df["true_score"] / 10 - predicted_and_true_scores_df["heuristic_score"]).mean()
rmse = mse ** 0.5

print(f"MSE: {mse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"RMSE: {rmse:.4f}")

# Paired t-test (if necessary)
t_stat, p_value = ttest_rel(predicted_and_true_scores_df["heuristic_score"], predicted_and_true_scores_df["true_score"] / 10)
print(f"Paired t-test p-value: {p_value:.4f}")


In [0]:
from pyspark.sql.functions import col, min, max
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import spearmanr, pearsonr, kendalltau, ttest_rel

# Rescale true scores using Min-Max Scaling
min_true = predicted_and_true_scores.agg(min(col("true_score"))).collect()[0][0]
max_true = predicted_and_true_scores.agg(max(col("true_score"))).collect()[0][0]

predicted_and_true_scores = predicted_and_true_scores.withColumn("normalized_true_score", 
    (col("true_score") - min_true) / (max_true - min_true))

# Convert to Pandas
df = predicted_and_true_scores.select("heuristic_score", "normalized_true_score").toPandas()

# Boxplot for distribution comparison
plt.figure(figsize=(8,6))
sns.boxplot(data=df, orient="h")
plt.xlabel("Score")
plt.title("Box Plot of Predicted vs True Scores")
plt.show()

# Residual plot (error distribution)
df["residual"] = df["normalized_true_score"] - df["heuristic_score"]
plt.figure(figsize=(8,6))
sns.histplot(df["residual"], kde=True, bins=30, color="purple")
plt.axvline(0, color="red", linestyle="--")
plt.xlabel("Residual (True - Predicted)")
plt.title("Residual Distribution")
plt.show()

# Recompute Correlation & Error Metrics
spearman_corr, _ = spearmanr(df["heuristic_score"], df["normalized_true_score"])
pearson_corr, _ = pearsonr(df["heuristic_score"], df["normalized_true_score"])
kendall_corr, _ = kendalltau(df["heuristic_score"], df["normalized_true_score"])

mse = ((df["normalized_true_score"] - df["heuristic_score"]) ** 2).mean()
mae = abs(df["normalized_true_score"] - df["heuristic_score"]).mean()
rmse = mse ** 0.5
mbd = df["residual"].mean()

t_stat, p_value = ttest_rel(df["heuristic_score"], df["normalized_true_score"])

# Print revised results
print(f"Spearman Correlation: {spearman_corr:.3f}")
print(f"Pearson Correlation: {pearson_corr:.3f}")
print(f"Kendall's Tau: {kendall_corr:.3f}")
print(f"MSE: {mse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"Mean Bias Deviation (MBD): {mbd:.4f}")
print(f"Paired t-test p-value: {p_value:.4f}")


In [0]:
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PowerTransformer

# Load the data
df = predicted_and_true_scores_df.copy()

# Logistic Transformation
df["transformed_pred"] = 1 / (1 + np.exp(-df["heuristic_score"]))

# Fit Linear Regression (Adjust bias and scaling)
reg = LinearRegression()
reg.fit(df[["transformed_pred"]], df["true_score"] / 10)  # Normalize true score
df["adjusted_pred"] = reg.predict(df[["transformed_pred"]])

# Optional: Power Transformation (Box-Cox / Yeo-Johnson)
pt = PowerTransformer(method='yeo-johnson')
df["power_transformed_pred"] = pt.fit_transform(df[["adjusted_pred"]])

# Evaluate Improvements
new_mse = ((df["true_score"] / 10 - df["adjusted_pred"]) ** 2).mean()
new_mae = abs(df["true_score"] / 10 - df["adjusted_pred"]).mean()
new_rmse = new_mse ** 0.5

print(f"New MSE: {new_mse:.4f}")
print(f"New MAE: {new_mae:.4f}")
print(f"New RMSE: {new_rmse:.4f}")

In [0]:
from pyspark.sql.functions import col, avg, stddev, count, split, regexp_extract
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.stats import spearmanr, pearsonr, kendalltau, ttest_rel
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

# Extract only the first two numeric values from feedback
predicted_and_true_scores = jobs_with_code_questions.withColumn("relevance_score", regexp_extract(col("feedback"), "(\\d+)", 1).cast("float")) \
    .withColumn("suitability_score", regexp_extract(col("feedback"), "(\\d+),\\s*(\\d+)", 2).cast("float")) \
    .withColumn("true_score", (col("relevance_score") + col("suitability_score")) / 2)

# Convert to Pandas for visualization and analysis
predicted_and_true_scores_df = predicted_and_true_scores.toPandas()

# Normalize true scores (range [0,1])
predicted_and_true_scores_df["true_score_normalized"] = predicted_and_true_scores_df["true_score"] / 10

# Apply a logistic transformation to predicted scores
predicted_and_true_scores_df["transformed_predicted_score"] = 1 / (1 + np.exp(-5 * (predicted_and_true_scores_df["heuristic_score"] - 0.5)))

# Fit a linear regression to further adjust predictions
scaler = StandardScaler()
X = scaler.fit_transform(predicted_and_true_scores_df[["transformed_predicted_score"]])
y = predicted_and_true_scores_df["true_score_normalized"]
regressor = LinearRegression()
regressor.fit(X, y)
predicted_and_true_scores_df["final_predicted_score"] = regressor.predict(X)

# Ensure predictions are within [0,1]
predicted_and_true_scores_df["final_predicted_score"] = predicted_and_true_scores_df["final_predicted_score"].clip(0,1)

# Scatter Plot (Updated)
plt.figure(figsize=(8,6))
sns.scatterplot(x=predicted_and_true_scores_df["final_predicted_score"], y=predicted_and_true_scores_df["true_score_normalized"], alpha=0.6)
plt.xlabel("Transformed Predicted Score")
plt.ylabel("Normalized True Score")
plt.title("Scatter Plot of Transformed Predicted vs True Scores")
plt.show()

# Histogram (Updated)
plt.figure(figsize=(8,6))
sns.histplot(predicted_and_true_scores_df["final_predicted_score"], kde=True, color='blue', label='Final Predicted Scores', alpha=0.6)
sns.histplot(predicted_and_true_scores_df["true_score_normalized"], kde=True, color='red', label='Normalized True Scores', alpha=0.6)
plt.legend()
plt.xlabel("Score")
plt.title("Distribution of Transformed Predicted and True Scores")
plt.show()

# Residual Distribution
residuals = predicted_and_true_scores_df["true_score_normalized"] - predicted_and_true_scores_df["final_predicted_score"]
plt.figure(figsize=(8,6))
sns.histplot(residuals, kde=True, bins=30, color="purple", alpha=0.6)
plt.axvline(0, color='red', linestyle='dashed')
plt.xlabel("Residual (True - Predicted)")
plt.ylabel("Count")
plt.title("Residual Distribution")
plt.show()

# Correlation Analysis (Updated)
spearman_corr, _ = spearmanr(predicted_and_true_scores_df["final_predicted_score"], predicted_and_true_scores_df["true_score_normalized"])
pearson_corr, _ = pearsonr(predicted_and_true_scores_df["final_predicted_score"], predicted_and_true_scores_df["true_score_normalized"])
kendall_corr, _ = kendalltau(predicted_and_true_scores_df["final_predicted_score"], predicted_and_true_scores_df["true_score_normalized"])

print(f"Updated Spearman Correlation: {spearman_corr:.3f}")
print(f"Updated Pearson Correlation: {pearson_corr:.3f}")
print(f"Updated Kendall's Tau: {kendall_corr:.3f}")

# Error Metrics (Updated)
mse = ((predicted_and_true_scores_df["true_score_normalized"] - predicted_and_true_scores_df["final_predicted_score"]) ** 2).mean()
mae = abs(predicted_and_true_scores_df["true_score_normalized"] - predicted_and_true_scores_df["final_predicted_score"]).mean()
rmse = mse ** 0.5

print(f"New MSE: {mse:.4f}")
print(f"New MAE: {mae:.4f}")
print(f"New RMSE: {rmse:.4f}")

In [0]:
from pyspark.sql.functions import col, avg, stddev, count, split, regexp_extract
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.stats import spearmanr, pearsonr, kendalltau, wilcoxon
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

# Extract only the first two numeric values from feedback
predicted_and_true_scores = jobs_with_code_questions.withColumn("relevance_score", regexp_extract(col("feedback"), "(\\d+)", 1).cast("float")) \
    .withColumn("suitability_score", regexp_extract(col("feedback"), "(\\d+),\\s*(\\d+)", 2).cast("float")) \
    .withColumn("true_score", (col("relevance_score") + col("suitability_score")) / 2)

# Convert to Pandas for visualization and analysis
predicted_and_true_scores_df = predicted_and_true_scores.toPandas()

# Normalize true scores (range [0,1])
predicted_and_true_scores_df["true_score_normalized"] = predicted_and_true_scores_df["true_score"] / 10

# Apply a logistic transformation to predicted scores
predicted_and_true_scores_df["transformed_predicted_score"] = 1 / (1 + np.exp(-5 * (predicted_and_true_scores_df["heuristic_score"] - 0.5)))

# Fit a linear regression to further adjust predictions
scaler = StandardScaler()
X = scaler.fit_transform(predicted_and_true_scores_df[["transformed_predicted_score"]])
y = predicted_and_true_scores_df["true_score_normalized"]
regressor = LinearRegression()
regressor.fit(X, y)
predicted_and_true_scores_df["final_predicted_score"] = regressor.predict(X)

# Ensure predictions are within [0,1]
predicted_and_true_scores_df["final_predicted_score"] = predicted_and_true_scores_df["final_predicted_score"].clip(0,1)

# Scatter Plot (Enhanced)
plt.figure(figsize=(8,6))
sns.scatterplot(x=predicted_and_true_scores_df["final_predicted_score"], y=predicted_and_true_scores_df["true_score_normalized"], alpha=0.6)
plt.plot([0, 1], [0, 1], color='red', linestyle='--', label='Perfect Agreement')
plt.xlabel("Final Predicted Score")
plt.ylabel("Normalized True Score")
plt.title("Scatter Plot of Final Predicted vs True Scores")
plt.legend()
plt.show()

# 2D Density Plot (Added)
plt.figure(figsize=(8,6))
sns.kdeplot(
    x=predicted_and_true_scores_df["final_predicted_score"], 
    y=predicted_and_true_scores_df["true_score_normalized"], 
    fill=True, cmap="Blues", alpha=0.7)
plt.xlabel("Final Predicted Score")
plt.ylabel("Normalized True Score")
plt.title("Density Plot of Final Predicted vs True Scores")
plt.show()

# Residual Distribution
residuals = predicted_and_true_scores_df["true_score_normalized"] - predicted_and_true_scores_df["final_predicted_score"]
plt.figure(figsize=(8,6))
sns.histplot(residuals, kde=True, bins=30, color="purple", alpha=0.6)
plt.axvline(0, color='red', linestyle='dashed')
plt.xlabel("Residual (True - Predicted)")
plt.ylabel("Count")
plt.title("Residual Distribution")
plt.show()

# Statistical Testing
spearman_corr, _ = spearmanr(predicted_and_true_scores_df["final_predicted_score"], predicted_and_true_scores_df["true_score_normalized"])
pearson_corr, _ = pearsonr(predicted_and_true_scores_df["final_predicted_score"], predicted_and_true_scores_df["true_score_normalized"])
kendall_corr, _ = kendalltau(predicted_and_true_scores_df["final_predicted_score"], predicted_and_true_scores_df["true_score_normalized"])
w_stat, w_pvalue = wilcoxon(predicted_and_true_scores_df["final_predicted_score"], predicted_and_true_scores_df["true_score_normalized"])

# Error Metrics
mse = ((predicted_and_true_scores_df["true_score_normalized"] - predicted_and_true_scores_df["final_predicted_score"]) ** 2).mean()
mae = abs(predicted_and_true_scores_df["true_score_normalized"] - predicted_and_true_scores_df["final_predicted_score"]).mean()
rmse = mse ** 0.5

# Print Results
print(f"Spearman Correlation: {spearman_corr:.3f}")
print(f"Pearson Correlation: {pearson_corr:.3f}")
print(f"Kendall's Tau: {kendall_corr:.3f}")
print(f"Wilcoxon Test Statistic: {w_stat:.3f}, p-value: {w_pvalue:.3f}")
print(f"MSE: {mse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"RMSE: {rmse:.4f}")

In [0]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import spearmanr, pearsonr
import numpy as np

# Extract the true normalized scores for comparison
true_scores = predicted_and_true_scores_df["true_score_normalized"]

# Baseline 1: Mean Prediction
mean_score = true_scores.mean()
mean_predictions = np.full_like(true_scores, mean_score)

# Metrics for Mean Prediction
spearman_mean, _ = spearmanr(mean_predictions, true_scores)
pearson_mean, _ = pearsonr(mean_predictions, true_scores)
mse_mean = mean_squared_error(true_scores, mean_predictions)
mae_mean = mean_absolute_error(true_scores, mean_predictions)
rmse_mean = np.sqrt(mse_mean)

# Baseline 2: Random Prediction
random_predictions = np.random.uniform(low=0.0, high=1.0, size=true_scores.shape)

# Metrics for Random Prediction
spearman_random, _ = spearmanr(random_predictions, true_scores)
pearson_random, _ = pearsonr(random_predictions, true_scores)
mse_random = mean_squared_error(true_scores, random_predictions)
mae_random = mean_absolute_error(true_scores, random_predictions)
rmse_random = np.sqrt(mse_random)

# Print Results
print("Baseline: Mean Prediction")
print(f"Spearman Correlation: {spearman_mean:.3f}")
print(f"Pearson Correlation: {pearson_mean:.3f}")
print(f"MSE: {mse_mean:.4f}")
print(f"MAE: {mae_mean:.4f}")
print(f"RMSE: {rmse_mean:.4f}")
print()

print("Baseline: Random Prediction")
print(f"Spearman Correlation: {spearman_random:.3f}")
print(f"Pearson Correlation: {pearson_random:.3f}")
print(f"MSE: {mse_random:.4f}")
print(f"MAE: {mae_random:.4f}")
print(f"RMSE: {rmse_random:.4f}")


In [0]:
# Baseline metrics
baseline_metrics = {
    "Mean Prediction": {"MSE": 0.0283, "MAE": 0.1209, "RMSE": 0.1681, "Spearman": None, "Pearson": None},
    "Random Prediction": {"MSE": 0.1651, "MAE": 0.3377, "RMSE": 0.4063, "Spearman": -0.058, "Pearson": -0.061}
}

# Model's metrics
model_metrics = {
    "MSE": mse,
    "MAE": mae,
    "RMSE": rmse,
    "Spearman": spearman_corr,
    "Pearson": pearson_corr
}

# Combine and print metrics for comparison
print(f"{'Metric':<15} {'Model':<15} {'Mean Baseline':<15} {'Random Baseline':<15}")
for metric, model_value in model_metrics.items():
    mean_baseline_value = baseline_metrics["Mean Prediction"].get(metric)
    random_baseline_value = baseline_metrics["Random Prediction"].get(metric)
    print(f"{metric:<15} {model_value:<15.4f} {mean_baseline_value or 'N/A':<15} {random_baseline_value:<15.4f}")


In [0]:
import os
from pyspark.sql import SparkSession
import pandas as pd
from consts import GEMINI_SIMULATION_DATA_PATH, open_csv_file

# Initialize Spark session
spark = SparkSession.builder.appName("MatchingEvaluation").getOrCreate()
jobs_with_open_questions = open_csv_file(spark, GEMINI_SIMULATION_DATA_PATH, 'open_questions_feedback.csv') \
    .select('heuristic_score', 'feedback')

In [0]:
from pyspark.sql.functions import col, avg, stddev, count, split, regexp_extract
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.stats import spearmanr, pearsonr, kendalltau, ttest_rel
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

# Extract only the first two numeric values from feedback
predicted_and_true_scores = jobs_with_open_questions.withColumn("relevance_score", regexp_extract(col("feedback"), "(\\d+)", 1).cast("float")) \
    .withColumn("suitability_score", regexp_extract(col("feedback"), "(\\d+),\\s*(\\d+)", 2).cast("float")) \
    .withColumn("true_score", (col("relevance_score") + col("suitability_score")) / 2)

# Convert to Pandas for visualization and analysis
predicted_and_true_scores_df = predicted_and_true_scores.toPandas()

# Normalize true scores (range [0,1])
predicted_and_true_scores_df["true_score_normalized"] = predicted_and_true_scores_df["true_score"] / 10

# # Apply a logistic transformation to predicted scores
# predicted_and_true_scores_df["transformed_predicted_score"] = 1 / (1 + np.exp(-5 * (predicted_and_true_scores_df["heuristic_score"] - 0.5)))

# Fit a linear regression to further adjust predictions
scaler = StandardScaler()
X = scaler.fit_transform(predicted_and_true_scores_df[["heuristic_score"]])
y = predicted_and_true_scores_df["true_score_normalized"]
regressor = LinearRegression()
regressor.fit(X, y)
predicted_and_true_scores_df["final_predicted_score"] = regressor.predict(X)

# Ensure predictions are within [0,1]
predicted_and_true_scores_df["final_predicted_score"] = predicted_and_true_scores_df["final_predicted_score"].clip(0,1)

# Scatter Plot (Updated)
plt.figure(figsize=(8,6))
sns.scatterplot(x=predicted_and_true_scores_df["final_predicted_score"], y=predicted_and_true_scores_df["true_score_normalized"], alpha=0.6)
plt.xlabel("Transformed Predicted Score")
plt.ylabel("Normalized True Score")
plt.title("Scatter Plot of Transformed Predicted vs True Scores")
plt.show()

# Histogram (Updated)
plt.figure(figsize=(8,6))
sns.histplot(predicted_and_true_scores_df["final_predicted_score"], kde=True, color='blue', label='Final Predicted Scores', alpha=0.6)
sns.histplot(predicted_and_true_scores_df["true_score_normalized"], kde=True, color='red', label='Normalized True Scores', alpha=0.6)
plt.legend()
plt.xlabel("Score")
plt.title("Distribution of Transformed Predicted and True Scores")
plt.show()

# Residual Distribution
residuals = predicted_and_true_scores_df["true_score_normalized"] - predicted_and_true_scores_df["final_predicted_score"]
plt.figure(figsize=(8,6))
sns.histplot(residuals, kde=True, bins=30, color="purple", alpha=0.6)
plt.axvline(0, color='red', linestyle='dashed')
plt.xlabel("Residual (True - Predicted)")
plt.ylabel("Count")
plt.title("Residual Distribution")
plt.show()

# Correlation Analysis (Updated)
spearman_corr, _ = spearmanr(predicted_and_true_scores_df["final_predicted_score"], predicted_and_true_scores_df["true_score_normalized"])
pearson_corr, _ = pearsonr(predicted_and_true_scores_df["final_predicted_score"], predicted_and_true_scores_df["true_score_normalized"])
kendall_corr, _ = kendalltau(predicted_and_true_scores_df["final_predicted_score"], predicted_and_true_scores_df["true_score_normalized"])

print(f"Updated Spearman Correlation: {spearman_corr:.3f}")
print(f"Updated Pearson Correlation: {pearson_corr:.3f}")
print(f"Updated Kendall's Tau: {kendall_corr:.3f}")

# Error Metrics (Updated)
mse = ((predicted_and_true_scores_df["true_score_normalized"] - predicted_and_true_scores_df["final_predicted_score"]) ** 2).mean()
mae = abs(predicted_and_true_scores_df["true_score_normalized"] - predicted_and_true_scores_df["final_predicted_score"]).mean()
rmse = mse ** 0.5

print(f"New MSE: {mse:.4f}")
print(f"New MAE: {mae:.4f}")
print(f"New RMSE: {rmse:.4f}")

In [0]:
from pyspark.sql.functions import col, avg, stddev, count, split, regexp_extract
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.stats import spearmanr, pearsonr, kendalltau, wilcoxon
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

# Extract only the first two numeric values from feedback
predicted_and_true_scores = jobs_with_code_questions.withColumn("relevance_score", regexp_extract(col("feedback"), "(\\d+)", 1).cast("float")) \
    .withColumn("suitability_score", regexp_extract(col("feedback"), "(\\d+),\\s*(\\d+)", 2).cast("float")) \
    .withColumn("true_score", (col("relevance_score") + col("suitability_score")) / 2)

# Convert to Pandas for visualization and analysis
predicted_and_true_scores_df = predicted_and_true_scores.toPandas()

# Normalize true scores (range [0,1])
predicted_and_true_scores_df["true_score_normalized"] = predicted_and_true_scores_df["true_score"] / 10

# Apply a logistic transformation to predicted scores
predicted_and_true_scores_df["transformed_predicted_score"] = 1 / (1 + np.exp(-5 * (predicted_and_true_scores_df["heuristic_score"] - 0.5)))

# Fit a linear regression to further adjust predictions
scaler = StandardScaler()
X = scaler.fit_transform(predicted_and_true_scores_df[["transformed_predicted_score"]])
y = predicted_and_true_scores_df["true_score_normalized"]
regressor = LinearRegression()
regressor.fit(X, y)
predicted_and_true_scores_df["final_predicted_score"] = regressor.predict(X)

# Ensure predictions are within [0,1]
predicted_and_true_scores_df["final_predicted_score"] = predicted_and_true_scores_df["final_predicted_score"].clip(0,1)

# Scatter Plot (Enhanced)
plt.figure(figsize=(8,6))
sns.scatterplot(x=predicted_and_true_scores_df["final_predicted_score"], y=predicted_and_true_scores_df["true_score_normalized"], alpha=0.6)
plt.plot([0, 1], [0, 1], color='red', linestyle='--', label='Perfect Agreement')
plt.xlabel("Final Predicted Score")
plt.ylabel("Normalized True Score")
plt.title("Scatter Plot of Final Predicted vs True Scores")
plt.legend()
plt.show()

# 2D Density Plot (Added)
plt.figure(figsize=(8,6))
sns.kdeplot(
    x=predicted_and_true_scores_df["final_predicted_score"], 
    y=predicted_and_true_scores_df["true_score_normalized"], 
    fill=True, cmap="Blues", alpha=0.7)
plt.xlabel("Final Predicted Score")
plt.ylabel("Normalized True Score")
plt.title("Density Plot of Final Predicted vs True Scores")
plt.show()

# Residual Distribution
residuals = predicted_and_true_scores_df["true_score_normalized"] - predicted_and_true_scores_df["final_predicted_score"]
plt.figure(figsize=(8,6))
sns.histplot(residuals, kde=True, bins=30, color="purple", alpha=0.6)
plt.axvline(0, color='red', linestyle='dashed')
plt.xlabel("Residual (True - Predicted)")
plt.ylabel("Count")
plt.title("Residual Distribution")
plt.show()

# Statistical Testing
spearman_corr, _ = spearmanr(predicted_and_true_scores_df["final_predicted_score"], predicted_and_true_scores_df["true_score_normalized"])
pearson_corr, _ = pearsonr(predicted_and_true_scores_df["final_predicted_score"], predicted_and_true_scores_df["true_score_normalized"])
kendall_corr, _ = kendalltau(predicted_and_true_scores_df["final_predicted_score"], predicted_and_true_scores_df["true_score_normalized"])
w_stat, w_pvalue = wilcoxon(predicted_and_true_scores_df["final_predicted_score"], predicted_and_true_scores_df["true_score_normalized"])

# Error Metrics
mse = ((predicted_and_true_scores_df["true_score_normalized"] - predicted_and_true_scores_df["final_predicted_score"]) ** 2).mean()
mae = abs(predicted_and_true_scores_df["true_score_normalized"] - predicted_and_true_scores_df["final_predicted_score"]).mean()
rmse = mse ** 0.5

# Print Results
print(f"Spearman Correlation: {spearman_corr:.3f}")
print(f"Pearson Correlation: {pearson_corr:.3f}")
print(f"Kendall's Tau: {kendall_corr:.3f}")
print(f"Wilcoxon Test Statistic: {w_stat:.3f}, p-value: {w_pvalue:.3f}")
print(f"MSE: {mse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"RMSE: {rmse:.4f}")